This notebook, unlike previous notebooks, uses the new shared saga code repo.  It also require py3.

# Imports/Setup 

In [126]:
import os
import sys
from glob import glob

import numpy as np

from astropy import units as u
from astropy import table
from astropy.coordinates import SkyCoord, UnitSphericalRepresentation, CartesianRepresentation, matrix_utilities

In [2]:
%matplotlib inline
from matplotlib import style, pyplot as plt

The below two cells build the saga repo and sets the path to look for it

In [3]:
startdir = !pwd
startdir = startdir[0]
%cd ../../saga
sagadir = !pwd
sagadir = sagadir[0]
!python setup.py build
%cd $startdir

/Users/erik/projects/saga/saga
running build
running build_py
copying SAGA/hosts/host_catalog.py -> build/lib/SAGA/hosts
/Users/erik/projects/saga/erik/ipython_notebooks


In [4]:
sagalibdir = os.path.join(sagadir, 'build/lib')
print('Adding', sagalibdir, 'to the python path')
sys.path.insert(0, sagalibdir)

import SAGA as saga

Adding /Users/erik/projects/saga/saga/build/lib to the python path


# Lists of hosts

In [5]:
db = saga.Database('/Users/erik/Dropbox/SAGA/')
hostcat = saga.HostCatalog(db)
hosttab = hostcat.load()

hosttab

NSAID,RA,Dec,PGC#,vhelio,distance,r,i,z,I,K,K_err,flag,K_abs,r_abs,b,dist_to_group,brightest_K_nearby,SDSS_FRACGOOD,RA_sub,Dec_sub,othername,vhelio_sub,vhelio_err,distance_sub,dist_2MASS_NSA,u,u_err,A_u,M_u,g,g_err,A_g,M_g,r_sub,r_err,A_r,M_r,i_sub,i_err,A_i,M_i,z_sub,z_err,A_z,M_z,J,J_err,A_J,M_J,H,H_err,A_H,M_H,K_sub,K_err_sub,A_K,M_K,stellar_mass,sersic_n,MK_compiled,SAGA_name,coord
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"deg,deg"
int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,str16,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str11,object
145729,224.594,-1.09102,53499,1922.0,28.4038,11.9155,11.457,11.0653,9.49,7.693,0.019,0,-24.5739,-20.3514,48.4266,3.55266,12.164,0.983605,224.594,-1.09102,g1458227-010528,1922.0,-1,28.4038,0.520533,14.1855,0.0109577,0.245277,-18.3542,12.3155,0.000563506,0.191118,-20.1622,11.2899,0.000420059,0.132215,-21.1207,10.7733,0.000416391,0.0982496,-21.6016,10.3333,0.000670495,0.0730797,-22.0151,8.594,0.02,0.0410241,-23.7147,7.951,0.023,0.02598,-24.3439,7.713,0.031,0.0174743,-24.5597,19800000000.0,2.82806,-24.5739,Othello,"224.594,-1.09102"
150887,348.683,4.53406,70795,2689.0,37.019,11.9764,11.6247,11.3038,10.57,8.327,0.013,0,-24.5151,-20.8657,-50.6545,5.73166,10.189,0.98742,348.683,4.53406,NGC_7541,2689.0,-1,37.019,0.844612,13.5013,0.00257254,0.285874,-19.6741,12.1673,0.000503632,0.222751,-20.9126,11.3808,0.000382114,0.154098,-21.6316,10.9634,0.000777402,0.114511,-22.004,10.589,0.0018768,0.0851754,-22.3494,9.439,0.005,0.0478142,-23.451,8.757,0.006,0.0302801,-24.1194,8.351,0.007,0.0203666,-24.4949,25100000000.0,0.777359,-24.5151,Catch22,"348.683,4.53406"
140594,180.098,-1.10008,37845,1465.0,23.2487,11.1116,10.7406,10.4707,9.35,7.321,0.015,0,-24.511,-20.7204,59.2146,2.29212,12.669,0.985217,180.098,-1.10008,g1200236-010600,1465.0,-1,23.2487,0.508479,12.5696,0.00122745,0.112339,-19.3871,11.1341,0.00028931,0.0875336,-20.795,10.3932,0.000242571,0.0605553,-21.5041,9.98552,0.000254046,0.0449991,-21.896,9.68725,0.000379465,0.0334711,-22.1818,8.271,0.009,0.0187894,-23.5789,7.604,0.012,0.0118991,-24.24,7.33,0.015,0.00800337,-24.5004,20600000000.0,1.7545,-24.511,Alice,"180.098,-1.10008"
142722,194.863,14.1712,44600,1988.0,30.027,11.4723,11.0534,10.7367,9.93,7.907,0.016,0,-24.4806,-20.9153,76.9088,3.38499,10.768,0.987676,194.863,14.1712,NGC_4866,1988.0,-1,30.027,0.860169,13.5959,0.0034416,0.117999,-18.9324,11.6346,0.000660792,0.0919442,-20.8639,10.7622,0.000601578,0.0636066,-21.6974,10.3064,0.000701003,0.0472665,-22.1363,9.95885,0.000769763,0.0351576,-22.4698,8.856,0.017,0.0197361,-23.5504,8.186,0.018,0.0124986,-24.2145,7.917,0.02,0.00840664,-24.466,35000000000.0,2.18747,-24.4806,,"194.863,14.1712"
150578,346.236,12.3229,70419,2381.0,33.313,11.5503,11.1597,10.903,9.89,8.162,0.021,0,-24.4511,-21.0628,-42.8417,9.44227,14.017,0.987084,346.236,12.3229,NGC_7479,2381.0,-1,33.313,0.271001,13.4744,0.00466526,0.477309,-19.6406,11.9191,0.000694477,0.371916,-21.0857,11.0737,0.000516905,0.257289,-21.8076,10.5901,0.000479616,0.191194,-22.2242,10.2512,0.000863628,0.142213,-22.5123,9.194,0.011,0.0798329,-23.4982,8.508,0.011,0.0505571,-24.1574,8.202,0.019,0.034005,-24.4297,34400000000.0,2.01064,-24.4511,,"346.236,12.3229"
163956,206.33,41.7125,48767,2549.01,37.2285,12.2853,11.7813,11.3937,11.43,8.434,0.012,0,-24.4204,-20.5691,71.714,9.76207,9.739,0.985273,206.33,41.7125,13451914+4142447,2549.01,-1,37.2285,0.879017,14.1416,0.00298302,0.0288247,-18.7735,12.4646,0.00131613,0.02246,-20.4361,11.5252,0.000345898,0.0155377,-21.3595,10.9973,0.00034377,0.0115462,-21.8814,10.5916,0.000769146,0.00858824,-22.2824,9.511,0.013,0.00482111,-23.3483

In [9]:
msk1 = 8*u.hourangle < hosttab['coord'].ra
msk2 = hosttab['coord'].ra < 18*u.hourangle
msk3 = hosttab['coord'].dec > -10*u.deg
msk4 = hosttab['SAGA_name'] != ''

binomsk = msk1 & msk2 & msk3
namedbinomsk = msk1 & msk2 & msk3 & msk4

np.sum(binomsk), np.sum(namedbinomsk)

(56, 10)

# Overlap with alternate imaging surveys 

## DECaLS

In [14]:
decals_path = os.path.join(os.environ['HOME'], 'astrodata', 'decals', 'dr5')

bricks_fn = os.path.join(decals_path, 'survey-bricks.fits.gz')
bricks_dr_fn = os.path.join(decals_path, 'survey-bricks-dr5.fits.gz')

In [15]:
bricks = table.Table.read(bricks_fn)
bricks_dr = table.Table.read(bricks_dr_fn)

In [195]:
bricks_in_dr = bricks[np.in1d(bricks['BRICKNAME'], bricks_dr['brickname'])]

In [206]:
def random_sample_cone(angle, n, ra0=0*u.deg, dec0=0*u.deg):
    phi = np.random.uniform(0, 2*np.pi, n)
    z = np.random.uniform(np.cos(angle), 1, n)
    s = np.sqrt(1-z**2)
    x = s * np.cos(phi)
    y = s * np.sin(phi)
    xyz = np.array([x, y, z])
    
    A = matrix_utilities.rotation_matrix(90*u.deg - dec0, 'y')
    B = matrix_utilities.rotation_matrix(180*u.deg - ra0, 'z')
    
    return CartesianRepresentation(B @ A @ xyz)

def host_overlap(hostrow, brickstocheck, environrad=300*u.kpc, precision=1e-2):
    host_ra = hostrow['coord'].ra
    host_dec = hostrow['coord'].dec
    host_distance = hostrow['distance']*u.Mpc
    
    if environrad.unit.is_equivalent(u.kpc):
        environrad = (environrad/host_distance).to(u.rad, u.dimensionless_angles())
    elif environrad.unit.is_equivalent(u.deg):
        pass
    else:
        raise u.UnitsError('invalid unit for environrad')
        
    n = int(np.ceil(1/precision))
    testpoints = random_sample_cone(environrad, n, host_ra, host_dec)
    testsc = SkyCoord(testpoints)
    
    msk_ra1 = brickstocheck['RA1'] < testsc.ra[:, np.newaxis].deg
    msk_ra2 = testsc.ra[:, np.newaxis].deg < brickstocheck['RA2']
    msk_dec1 = brickstocheck['DEC1'] < testsc.dec[:, np.newaxis].deg
    msk_dec2 = testsc.dec[:, np.newaxis].deg < brickstocheck['DEC2']
    match = msk_ra1 & msk_ra2 & msk_dec1 & msk_dec2
    
    havepoint = np.any(match, axis=0)
    
    return match

In [212]:
nbricks = []
frac = []
for row in hosttab[binomsk]:
    print('on', row['NSAID'], row['SAGA_name'])
    brick_match = host_overlap(row, bricks_in_dr)
    nbricks.append(np.sum(np.any(brick_match, axis=0)))
    frac.append(np.sum(np.any(brick_match, axis=1))/brick_match.shape[0])
    print('nbricks:', nbricks[-1], 'frac:', frac[-1])
np.sum(np.array(frac)>.9)/len(frac), len(frac)

on 145729 Othello
nbricks: 25 frac: 1.0
on 140594 Alice
nbricks: 34 frac: 1.0
on 142722 
nbricks: 22 frac: 1.0
on 163956 
nbricks: 0 frac: 0.0
on 165153 
nbricks: 0 frac: 0.0
on 135879 
nbricks: 0 frac: 0.0
on 144151 
nbricks: 0 frac: 0.0
on 163136 
nbricks: 15 frac: 1.0
on 164995 
nbricks: 0 frac: 0.0
on 147100 Odyssey
nbricks: 17 frac: 1.0
on 140458 
nbricks: 16 frac: 1.0
on 147606 
nbricks: 16 frac: 1.0
on 144121 
nbricks: 0 frac: 0.0
on 138201 
nbricks: 29 frac: 1.0
on 135440 
nbricks: 19 frac: 1.0
on 143856 
nbricks: 0 frac: 0.0
on 52773 
nbricks: 0 frac: 0.0
on 160302 
nbricks: 0 frac: 0.0
on 139467 
nbricks: 13 frac: 1.0
on 157374 
nbricks: 21 frac: 1.0
on 165980 
nbricks: 0 frac: 0.0
on 138642 
nbricks: 27 frac: 1.0
on 158541 
nbricks: 0 frac: 0.0
on 165082 
nbricks: 29 frac: 1.0
on 145398 
nbricks: 28 frac: 1.0
on 166313 Gilgamesh
nbricks: 26 frac: 1.0
on 166141 
nbricks: 31 frac: 1.0
on 69028 
nbricks: 0 frac: 0.0
on 141465 
nbricks: 19 frac: 1.0
on 145879 
nbricks: 22 frac: 

(0.5178571428571429, 56)

In [213]:
nbricks = []
frac = []
for row in hosttab[namedbinomsk]:
    print('on', row['NSAID'], row['SAGA_name'])
    brick_match = host_overlap(row, bricks_in_dr)
    nbricks.append(np.sum(np.any(brick_match, axis=0)))
    frac.append(np.sum(np.any(brick_match, axis=1))/brick_match.shape[0])
    print('nbricks:', nbricks[-1], 'frac:', frac[-1])
np.sum(np.array(frac)>.9)/len(frac), len(frac)

on 145729 Othello
nbricks: 24 frac: 1.0
on 140594 Alice
nbricks: 31 frac: 1.0
on 147100 Odyssey
nbricks: 20 frac: 1.0
on 166313 Gilgamesh
nbricks: 21 frac: 1.0
on 13927 Sopranos
nbricks: 22 frac: 1.0
on 165536 Dune
nbricks: 29 frac: 1.0
on 33446 StarTrek
nbricks: 0 frac: 0.0
on 85746 MobyDick
nbricks: 29 frac: 1.0
on 161174 ScoobyDoo
nbricks: 17 frac: 1.0
on 137625 Oz
nbricks: 33 frac: 1.0


(0.90000000000000002, 10)

## HSC 

In [44]:
import healpy

hscmap, hschead = healpy.read_map('../S16A_wide2_fdfc_i_limitmag_hp_frac_cut_0.02.fits', h=True)
hschead = dict(hschead)  #This is a VERY foolish healpy design choice!!

NSIDE = 2048
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING


In [48]:
def query_disc_astropy(cen, rad, nside=hschead['NSIDE'], **kwargs):
    uvec = cen.represent_as(UnitSphericalRepresentation).represent_as(CartesianRepresentation).xyz
    return healpy.query_disc(nside, uvec, rad.to(u.radian).value)

In [52]:
nametofrac_environs = {}
nametofrac_1deg = {}
for row in hosttab:
    cen = row['coord']
    rad = ((300*u.kpc)/(row['distance']*u.Mpc)).to(u.arcmin, u.dimensionless_angles())
    name = row['SAGA_name']
    if name == '':
        name = 'NSA' + str(row['NSAID'])
    
    host_pix = query_disc_astropy(cen, rad)
    nametofrac_environs[name]= np.sum(hscmap[host_pix])/len(host_pix)
    
useful_nametofrac_environs = {k:v for k,v in nametofrac_environs.items() if v != 0}
useful_nametofrac_environs

{'Alice': 0.75620229007633588, 'Dune': 1.0, 'Othello': 0.66309778729478941}

Note that *only* Alice has real data - the rest is "planned"